In [1]:
import torch 
from transformers import AutoTokenizer, AutoModel, AutoConfig
import pandas as pd
import pickle

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
data_dir='/home/mahbub/Desktop/sepsis_text_IoMT_project/data/embedding_generation_data/'

In [4]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to(device)
config = AutoConfig.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", output_hidden_states=True, hidden_size=768)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [6]:
with open(data_dir+'non_text_row_id_mapping_text_row_id_dict.pkl', 'rb') as f:
    non_text_row_id_mapping_text_row_id_dict = pickle.load(f)
    

In [7]:
sepsis_text = pd.read_csv(data_dir+'sepsis_text.csv')

/home/mahbub/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
token_vecs_sum = []
token_vecs_cat = []
long_embedding_dict = dict()
short_embedding_dict = dict()
count=0
total = len(non_text_row_id_mapping_text_row_id_dict)

for index in non_text_row_id_mapping_text_row_id_dict:
    
    count+=1
    
    text_row_ids = non_text_row_id_mapping_text_row_id_dict[index][0]
    row_text_list=sepsis_text.loc[sepsis_text['ROW_ID'].isin(text_row_ids)]['TEXT'].tolist()
    print(len(row_text_list))
    
    if len(row_text_list) == 0:
        
        long_sentence_embedding=torch.zeros(3072).to(device)
        short_sentence_embedding=torch.zeros(768).to(device)
        
        print('here')
        
    else:
        
        batch = tokenizer(row_text_list, padding=True, truncation=True, return_tensors="pt", max_length = 512).to(device)
        
        with torch.no_grad():
            
            outputs = model(batch['input_ids'], batch['token_type_ids'], output_hidden_states=True)
            
        hidden_states= outputs.hidden_states
        
        
        
        token_embeddings = torch.stack(hidden_states, dim=0)
        
        print(token_embeddings.shape)
        
        row_token_embeddings = torch.mean(token_embeddings, 1, True)
        
        print(row_token_embeddings.shape)
        
        token_embeddings = torch.squeeze(row_token_embeddings, dim=1)
        
        print(token_embeddings.shape)
        
        token_embeddings = token_embeddings.permute(1,0,2)
        
        print(token_embeddings.shape)
        
        # Stores the token vectors, with shape [512 x 768]
        token_vecs_sum = []
        
        # Stores the token vectors, with shape [512x 3,072]
        token_vecs_cat = []

        # `token_embeddings` is a [512x 12 x 768] tensor.

        # For each token in the sentence...
        for token in token_embeddings:

            # `token` is a [13 x 768] tensor

            # Sum the vectors from the last four layers. `sum_vec` is a [768] tensor
            sum_vec = torch.sum(token[-4:], dim=0)
            
            #print(sum_vec.shape)
    
            # Use `sum_vec` to represent `token`.
            token_vecs_sum.append(sum_vec)
        
            # Concatenate the vectors (that is, append them together) from the last 
            # four layers.
            # Each layer vector is 768 values, so `cat_vec` is length 3,072.
            cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
            #print(cat_vec.shape)
    
            # Use `cat_vec` to represent `token`.
            token_vecs_cat.append(cat_vec)

            #print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
            #print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))
        
        long_sentence_embedding = torch.mean(torch.stack(token_vecs_cat), dim=0)
        short_sentence_embedding = torch.mean(torch.stack(token_vecs_sum), dim=0)
            
        print(torch.stack(token_vecs_cat).shape)
        
    break
    #long_embedding_dict[index] = long_sentence_embedding
    #short_embedding_dict[index] = short_sentence_embedding
    
    #print(str(count)+'/'+str(total)+' is done.')

#torch.save(long_embedding_dict, 'torch_long_embedding_dict')
#torch.save(short_embedding_dict, 'torch_short_embedding_dict')
#loaded = torch.load('torch_db')

5
Ignored unknown kwarg option direction
torch.Size([13, 5, 512, 768])
torch.Size([13, 1, 512, 768])
torch.Size([13, 512, 768])
torch.Size([512, 13, 768])
torch.Size([512, 3072])
